# My Tutor: End of Week 1 Exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

## Setup

### Imports

In [5]:
import os
import requests
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

### Constants

In [ ]:
MODEL_GPT = "gpt-4o-mini"

OLLAMA_API = "http://localhost:11434/api/chat"
LL_HEADERS = {"Content-Type": "application/json"}
MODEL_LLAMA = "llama3.2"

### Setup Environment

In [3]:
# OpenAI
load_dotenv(override=True)
api_key = os.environ['OPENAI_API_KEY']

# Check key
if not api_key:
    print("No API Key found.")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start with sk-proj-")
elif api_key.strip() != api_key:
    print("An API key was found, but it was a space or a tab.")
else:
    print("API key found and looks good so far!")

openai = OpenAI()

API key found and looks good so far!


## Question

In [4]:
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

## Prompts

In [ ]:
system_prompt = "You are an assistant that takes technical questions and responds with an explanation, like a tutor. Respond in markdown."

In [7]:
system_prompt

'You are an assistant that takes technical questions and responds with an explanation. Respond in markdown.'

In [12]:
def user_prompt_question(question):
    user_prompt = "You are a tutor assistant. Please provide an answer and explanation in markdown. \nThe question is as follows: \n\n"
    user_prompt += question
    return user_prompt

In [13]:
print(user_prompt_question(question))

You are a tutor assistant. Please provide an answer and explanation in markdown. 
The question is as follows: 


Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}



## gpt-4o-mini (w streaming)

In [16]:
def stream_gpt_tutor(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt_question(question)
            }
        ],
        stream=True
    )

    # # stream text only first
    # for chunk in stream:
    #     print(chunk.choices[0].delta.content or '', end='')

    # Stream w MD
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [17]:
# markdown
stream_gpt_tutor(question)

Certainly! Let’s break down the code snippet:

python
yield from {book.get("author") for book in books if book.get("author")}


### Explanation

1. **Context of `yield from`:**
   - The `yield from` statement is used within a generator function in Python. It allows a generator to delegate part of its operations to another generator. This is useful for combining multiple generators or yielding values from another iterable.

2. **Set Comprehension:**
   - The construct `{book.get("author") for book in books if book.get("author")}` is a set comprehension. 
   - This means that it is an expression that creates a set (a collection of unique items) by iterating over the iterable `books`.

3. **Iteration Over `books`:**
   - The code iterates over a collection called `books`. Each `book` in `books` is expected to be a dictionary (or an object that supports the `get` method).

4. **Retrieving Authors:**
   - `book.get("author")` attempts to retrieve the value associated with the key `"author"` from the `book` dictionary.
   - If the key `"author"` does not exist, `book.get("author")` will return `None` (or the specified default value, if provided).

5. **Filtering Non-None Values:**
   - The condition `if book.get("author")` ensures that only those authors that are not `None` (valid authors) are included in the resulting set.
   - This effectively filters out any `book` entries that do not have a valid `"author"` key.

6. **Final Output:**
   - The result of the set comprehension is a set of unique authors from the `books` list.
   - By using `yield from` with this set, the generator will yield each unique author one at a time when iterated over.

### Why Use This Code?

- **Uniqueness of Authors:** Using a set means that each author will be listed only once, avoiding duplicates.
- **Efficiency:** The code combines the filtering and extraction of authors into a single line, making it both concise and readable.
- **Generator Context:** Using `yield from` allows the function to yield each author in a way that can be easily consumed by any calling function, respecting the generator's nature of yielding values over returning a complete collection.

### Summary

In summary, this code snippet effectively creates a generator that yields unique authors from a list of book dictionaries, ensuring that only valid authors are included. The use of `yield from` makes it possible to seamlessly integrate these yielded values into a larger generator flow.

In [ ]:
# Text only
stream_gpt_tutor(question)

Certainly! Let's break down the provided code snippet:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Code Explanation

1. **Comprehension**:
   - The code uses a **set comprehension**: `{ ... for ... in ... if ... }`.
   - Specifically, it iterates over a collection called `books`.

2. **Iteration**:
   - For each `book` in the `books` collection, the expression `book.get("author")` retrieves the value associated with the key `"author"` from the dictionary `book`.
   - `book.get("author")` returns the author's name if the key exists and is associated with a value. If the key does not exist, it returns `None`.

3. **Filtering**:
   - The `if book.get("author")` condition filters out books where the author's name is `None` or does not exist. So only books that have a valid author will be included in the results.

4. **Set Creation**:
   - The expression `{ ... }` gathers all unique authors (because a set inherently only keeps unique values).


## Llama 3.2

In [18]:
def llama_tutor(question):
    response = ollama.chat(
        model=MODEL_LLAMA,
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt_question(question)
            }
        ]
    )
    result = response['message']['content']
    display(Markdown(result))
    

In [19]:
llama_tutor(question)

**Explanation of the Code**
=====================================

### Overview

This line of code uses a combination of Python's iterator protocol, generator expressions, and the `yield from` syntax to fetch a list of authors from a collection of books.

### Breakdown

```markdown
yield from {book.get("author") for book in books if book.get("author")}
```

*   **Dictionary comprehension**: `{book.get("author") for book in books if book.get("author")}` creates an iterator that yields the values associated with the key `"author"` for each dictionary (`book`) in the `books` collection.
    *   The `if` condition filters out any dictionaries that do not have a key named `"author"`.
*   **Generator expression**: This is equivalent to saying "generate this value and remember where we left off so we can pick up again later." It allows us to use iteration instead of explicit looping.

### Why `yield from`

The `yield from` keyword is used in Python 3.3 onwards. When used with a generator or iterator expression, it yields the values yielded by the inner iterable (in this case, the dictionary comprehension).

Here's what happens when you call the above code:

1.  The dictionary comprehension `{book.get("author") for book in books if book.get("author")}` generates an iterator that iterates over each book in `books` and yields its author.
2.  For each book, it checks whether `"author"` exists as a key in the book's dictionary using `book.get("author")`. If present, it includes the value associated with `"author"`.
3.  The `yield from` statement then uses this iterator to yield its values.

### Equivalent Code

You can write equivalent code using explicit loops:

```python
def get_authors(books):
    authors = []
    for book in books:
        if "author" in book:
            authors.append(book["author"])
    return authors

# Using a generator expression (same as the original)
def get_authors_generator(books):
    return (book.get("author") for book in books if book.get("author"))

# Using yield from
def get_authors_yield_from(books):
    for book in books:
        if "author" in book:
            yield book["author"]
```

However, the original code with `yield from` is more concise and can be more memory-efficient because it avoids creating unnecessary lists or intermediate data structures.